
#**Pandemonium Project**

### JSON Parsing methods:

In [4]:
import sim
import json
import random
import networkx as nx
import numpy as np
import pandas as pd
import os
import shutil
import time
from dataclasses import dataclass
from google.colab import files


# Returns the round_type ('RR' or 'J' ), the num_seeds, and unique_id
def parse_round_specs(file):
  file_info = file.split('.')
  round_type = file_info[0].split('/')[-1]
  num_seeds = file_info[1]
  graph_id = file_info[2]
  return round_type, num_seeds, graph_id

# Returns the adjacency lists and round specifications
def parse_JSON_file(filename):
  adjacency_dict = {}
  with open(filename, 'r') as f:
    round_type, num_seeds, graph_id = parse_round_specs(filename)
    adjacency_dict = json.loads(f.read())
  return adjacency_dict, num_seeds, round_type, graph_id

# Returns the dictionary representation of the graph adjacencies
def parse_JSON_file_dict(filename):
  adjacency_dict = {}
  with open(filename, 'r') as f:
    _, num_seeds, _ = parse_round_specs(filename)
    adjacency_dict = json.loads(f.read())
  return adjacency_dict, int(num_seeds)

# Returns the filenames in the graphs folder
def parse_filenames(path):
  if not os.path.exists(path):
    os.mkdir(path)
  filenames = []
  for f in os.listdir(path):
    if not f.startswith('.'):
      filenames.append(path + f)
  return filenames

# Convert the output seeds into a file in the format for sim.py
def convert_to_dict_file(seeds, round_type, num_seeds, graph_id):
  plague_file = {}
  plague_file["plague_inc"] = []
  for i in range(50):
    plague_file["plague_inc"].append(seeds)
  write_filename = f'/content/test/{round_type}.{num_seeds}.{graph_id}-plague_inc.json'
  f = open(write_filename, "a")
  f.write(json.dumps(plague_file))
  f.close()
  return plague_file


### Strategy comparison functions:
---

In [5]:
# Compares two Round-Robin strategies and determines the winner
def compare_strategies(filename, seed_file, opponent_seed_file):
  adj_list, num_seeds = parse_JSON_file_dict(filename)
  node_mappings = {}

  with open(seed_file, 'r') as f:
    seed = json.loads(f.read())
    node_mappings['plague_inc'] = seed['plague_inc'][0]

  with open(opponent_seed_file, 'r') as f:
    seed = json.loads(f.read())
    key = list(seed.keys())[0]
    node_mappings[key] = seed[key][0]
  return sim.run(adj_list, node_mappings)

# Compares Jungle strategies amonst all participants and determines the winner
def compare_strategies_J(filename, seed_file, opponent_seed_folder):
  opponent_seed_files = parse_filenames(opponent_seed_folder)
  adj_list, num_seeds = parse_JSON_file_dict(filename)
  node_mappings = {}

  with open(seed_file, 'r') as f:
    seed = f.read().rstrip()
    node_mappings['plague_inc'] = seed.split('\n')[:num_seeds]

  for i, opponent_seed_file in enumerate(opponent_seed_files):
    with open(opponent_seed_file, 'r') as f:
      seed = json.loads(f.read())
      opponent_name = opponent_seed_file.split('-')[1].split('.')[0]
      node_mappings[opponent_name] = seed[next(iter(seed))][0]
  return sim.run(adj_list, node_mappings)

# Compares two nodes' performance during a sim.py run
def compare_nodes(filename, node, opponent):
  adj_list, num_seeds = parse_JSON_file_dict(filename)
  node_mappings = {}
  node_mappings['plague_inc'] = [node]
  node_mappings['opponent'] = [opponent]
  result = sim.run(adj_list, node_mappings)

  return result['plague_inc'] >= result['opponent']

### Algorithm for finding the seed nodes:

##### For Round Robin: Sort the highest degree nodes and run sim.py to find the num_seeds highest performing nodes.

##### For Jungle: Pick num_seeds random nodes from the 3rd set of highest degree nodes to the 7th set of highest degree nodes. (Each set being num_seeds length).
---

In [ ]:
# Retrieves the sorted highest degree nodes in the graph
filename = parse_filenames('/content/graphs/')[0]
adjacency_list, num_seeds, round_type, graph_id = parse_JSON_file(filename)
G = nx.Graph(adjacency_list)
sorted_degree = sorted(dict(G.degree).items(), key=lambda x: x[1], reverse=True)
seeds = np.array(sorted_degree)[:int(num_seeds), 0]
print(f"Top {int(num_seeds)} nodes: " + str(seeds))

if round_type == 'RR':
  # Round-Robin Strategy
  start_time = time_sec = time.time()
  index = 5 # Starting index for num_seeds = 10 -> 5, num_seeds = 15 -> 3, otherwise set to 2
  while time.time() - start_time < 240 and index <= 8:
    print("Time elapsed: " + str(time.time()- start_time))
    highest_deg_nodes = np.array(sorted_degree)[:int(num_seeds)*index, 0]
    print(f"Choosing the best seeds from the top {int(num_seeds) * index} nodes: " + str(highest_deg_nodes))
    # Compares each node in highest_deg_nodes with eachother using sim.py to find the best num_seed nodes
    node_competition = {node: 0 for node in highest_deg_nodes}
    for i in range(len(highest_deg_nodes)-1):
      for j in range(i+1, len(highest_deg_nodes)):
        if time.time() - start_time > 240:
          print("Time elapsed: " + str(time.time()- start_time))
          break
        result = compare_nodes(filename, highest_deg_nodes[i], highest_deg_nodes[j])
        node_competition[highest_deg_nodes[i]] += result
        node_competition[highest_deg_nodes[j]] += not result

    if time.time() - start_time <= 240:
      sorted_nodes = sorted(node_competition.items(), key=lambda x: x[1], reverse=True)
      seeds = np.array(sorted_nodes)[:int(num_seeds), 0]
      print(f"{index}: " + str(seeds))
      index += 1
else:
  # Jungle Strategy
  highest_deg_nodes = np.array(sorted_degree)[int(num_seeds)*2:int(num_seeds)*6, 0]
  seeds = random.sample(sorted(highest_deg_nodes), int(num_seeds))
  print(seeds)

### Seed file generation and download functions:
---



In [ ]:
# Generate the txt files for submitting seeds to the Pandemaniac website
def generate_txt_files(output_path, round_type, num_seeds, graph_id, seeds):
  rounds = 50
  if os.path.exists(output_path):
    shutil.rmtree(output_path)
  os.mkdir(output_path)
  file_id = 'plague_inc'
  write_filename = f'{output_path}{round_type}.{num_seeds}.{graph_id}-{file_id}.txt'
  f = open(write_filename, "a")
  for round in range(rounds):
    for seed in seeds:
      f.writelines(str(seed) + "\n")
  f.close()
  print(f'Generated file: {write_filename}')

# Download the txt files in the output directory
def download_files(output_dir):
  for filename in os.listdir(output_dir):
    files.download(output_dir + filename)

generate_txt_files('/content/output/', round_type, num_seeds, graph_id, seeds)
download_files('/content/output/')
# convert_to_dict_file(np.array(seeds).tolist(), round_type, num_seeds, graph_id)

### Strategy testing against opponent teams using sim.py:

---



In [ ]:
# filename = parse_filenames('/content/graphs/')[0]
# seed_file = parse_filenames('/content/test/')[0]
# round_type, _, _ = parse_round_specs(filename)
# if round_type == 'RR':
#   for fname in parse_filenames('/content/strats/'):
#     opponent_seed_file = f'{fname}'
#     result = compare_strategies(filename, seed_file, opponent_seed_file)
#     print(result)
# elif round_type == 'J':
#   opponent_seed_folder = '/content/opponent_strategies/' # Add all opponent strategies to this folder
#   results = sorted(compare_strategies_J(filename, seed_file, opponent_seed_folder).items(), key=lambda x: x[1], reverse=True)
#   for team in results:
#     print(f'{team[0]}: {team[1]}')